In [ ]:
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import stats
from ebnmpy.estimators import estimators

In [ ]:
def sample_point_normal(n, pi0=.9, mu=0, sigma=2):
    not_delta = stats.bernoulli.rvs(pi0, size=n) == 0
    z = np.full(n, mu, dtype=float)
    z[not_delta] = stats.norm.rvs(mu, sigma, size=not_delta.sum())
    return z

def sample_point_t(n, pi0=.8, df=5, scale=1.5):
    not_delta = stats.bernoulli.rvs(pi0, size=n) == 0
    z = np.zeros(n)
    z[not_delta] = stats.t.rvs(df=df, scale=scale, size=not_delta.sum())
    return z

def sample_assymetric_tophat(n, pi0=.5, a=-5, b=10):
    not_delta = stats.bernoulli.rvs(pi0, size=n) == 0
    z = np.zeros(n)
    z[not_delta] = stats.uniform.rvs(a, b - a, size=not_delta.sum())
    return z

def get_rmse(theta, theta_hat):
    return np.sqrt(np.mean((theta_hat - theta) ** 2))

def get_clcov(theta, samples, intervals=(.05, .95)):
    lower = np.quantile(samples, intervals[0], axis=0)
    upper = np.quantile(samples, intervals[1], axis=0)
    return ((theta >= lower) & (theta <= upper)).mean()

In [ ]:
s = 1
n = 1000
n_posterior_samples = 1001
n_simulations = 10

In [ ]:
np.random.seed(0)

In [ ]:
samplers = {
    "Point-normal": sample_point_normal,
    "Point-t": sample_point_t,
    "Asymmetric tophat": sample_assymetric_tophat,
}

results = []

for _ in range(n_simulations):
    for sampler_name, sampler in samplers.items():
        theta = sampler(n)
        x = theta + stats.norm.rvs(size=n)

        for cls_name, cls in estimators.items():
            est = cls(include_posterior_sampler=True).fit(x=x, s=s)
            samples = est.sample(n_posterior_samples)

            loglik = est.log_likelihood_
            rmse = get_rmse(theta, theta_hat=est.posterior_["mean"])
            clcov = get_clcov(theta, samples)

            results.append((sampler_name, cls.__name__, loglik, rmse, clcov))

In [ ]:
df = pd.DataFrame(results, columns=("Distribution", "Class", "LogLik", "RMSE", "ClCov"))
columns = list(itertools.product(list(samplers), ("LogLik", "RMSE", "ClCov")))
df_mean = df.groupby(["Distribution", "Class"]).mean().unstack(0).swaplevel(0, 1, axis=1)[columns].loc[[i.__name__ for i in estimators.values()]]

In [ ]:
df_mean.index.name = None
df_mean.columns.names = [None, None]

In [ ]:
formatter = {i: "{:.1f}" if "LogLik" in i else "{:.3f}" for i in columns}
s = df_mean.style.format(formatter=formatter)
s = s.background_gradient(cmap="Reds_r", subset=columns[::3]).background_gradient(cmap="Reds", subset=columns[1::3]).background_gradient(cmap="Reds_r", subset=columns[2::3])
s = s.set_properties(**{'text-align': 'center'})
s = s.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
for i in (3, 6):
    s = s.set_table_styles({
        columns[i]: [{'selector': 'th', 'props': 'border-left: 1px solid black'},
                                   {'selector': 'td', 'props': 'border-left: 1px solid #000000'}]
    }, overwrite=False, axis=0)

In [ ]:
s